# Azure AI Foundry Agent Service

This notebook demonstrates how to create and use agents with Azure AI Foundry Agent Service.

## Prerequisites
- Azure AI Foundry project created
- Azure OpenAI resource connected to your project
- Project credentials configured in `.env.local`

## Features
- ✅ Create AI agents with custom instructions
- ✅ Attach tools/functions to agents
- ✅ Manage conversations with threads
- ✅ Run agents and get responses
- ✅ Support for file search and code interpreter

## Table of Contents

1. [Setup and Installation](#setup-and-installation)
2. [Initialize Azure AI Project Client](#initialize-azure-ai-project-client)
3. [Example 1: Create a Simple Agent](#example-1-create-a-simple-agent)
4. [Create a Thread (Conversation)](#create-a-thread-conversation)
5. [Send a Message and Run the Agent](#send-a-message-and-run-the-agent)
6. [Example 2: Agent with Function Calling](#example-2-agent-with-function-calling)
7. [Example 3: Streaming Responses](#example-3-streaming-responses)
8. [Cleanup: Delete Agent and Threads](#cleanup-delete-agent-and-threads)
9. [Example 4: Using AgentManager Utility](#example-4-using-agentmanager-utility)
10. [Summary](#summary)

## Setup and Installation

In [ ]:
# Install required packages
%pip install -qU azure-ai-projects azure-identity azure-ai-inference python-dotenv

In [ ]:
# Check Azure AI Projects SDK version
import azure.ai.projects
print(f"Azure AI Projects SDK version: {azure.ai.projects.__version__}")

In [ ]:
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import FunctionTool, ToolSet

# Load environment variables from .env
load_dotenv("../.env")

print("✅ Imports successful")

## Configure PATH for Azure CLI

Ensure the Azure CLI is accessible in the notebook kernel's PATH.

In [ ]:
import os
import shutil

new_path_entry = "/opt/homebrew/bin"  # Replace with the directory you want to add
current_path = os.environ.get('PATH', '')

if new_path_entry not in current_path.split(os.pathsep):
    os.environ['PATH'] = new_path_entry + os.pathsep + current_path
    print(f"Updated PATH for this session: {os.environ['PATH']}")
else:
    print(f"PATH already contains {new_path_entry}: {current_path}")

# You can then verify with shutil.which again
print(f"Location of 'az' found by kernel now: {shutil.which('az')}")

In [ ]:
import shutil

print("Kernel's PATH:")
print(os.environ.get('PATH'))

print("\nLocation of 'az' found by kernel:")
print(shutil.which('az'))

## Initialize Azure AI Project Client

In [ ]:
# Get project endpoint
endpoint = os.getenv("AZURE_AI_PROJECT_ENDPOINT")

if not endpoint:
    raise ValueError("Please set AZURE_AI_PROJECT_ENDPOINT in .env.local")

# Initialize client with Azure credential (following official SDK documentation)
project_client = AIProjectClient(
    endpoint=endpoint,
    credential=DefaultAzureCredential()
)

print("✅ Azure AI Project Client initialized")
print(f"📦 Endpoint: {endpoint}")

## Example 1: Create a Simple Agent

In [ ]:
# Create an agent
agent = project_client.agents.create_agent(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o"),
    name="MyAssistant",
    instructions="You are a helpful AI assistant that answers questions concisely and accurately."
)

print(f"✅ Agent created: {agent.id}")
print(f"📝 Name: {agent.name}")
print(f"🤖 Model: {agent.model}")

## Create a Thread (Conversation)

In [ ]:
# Create a thread for the conversation
thread = project_client.agents.threads.create()

print(f"✅ Thread created: {thread.id}")

## Send a Message and Run the Agent

In [ ]:
# Add a message to the thread
message = project_client.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is Azure AI Foundry?"
)

print(f"📨 Message sent: {message.id}")

# Run the agent
run = project_client.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id
)

print(f"\n🤖 Agent Response:")
print("=" * 80)

# Get the messages
messages = project_client.agents.messages.list(thread_id=thread.id)

# Display the assistant's response
for msg in messages:
    if msg.role == "assistant":
        if msg.text_messages:
            for text_msg in msg.text_messages:
                print(text_msg.text.value)
        break

print("=" * 80)

## Example 2: Agent with Function Calling

In [ ]:
import json

# Define functions
def get_weather(location: str) -> str:
    """Get weather information for a location"""
    # Simulated weather data
    weather_data = {
        "San Francisco": {"temperature": 72, "condition": "sunny"},
        "Seattle": {"temperature": 58, "condition": "cloudy"},
        "New York": {"temperature": 65, "condition": "partly cloudy"}
    }
    
    data = weather_data.get(location, {"temperature": 70, "condition": "unknown"})
    return json.dumps(data)

def get_stock_price(symbol: str) -> str:
    """Get stock price for a symbol"""
    # Simulated stock data
    stock_data = {
        "MSFT": {"price": 380.50, "change": "+2.3%"},
        "AAPL": {"price": 195.30, "change": "-1.2%"},
        "GOOGL": {"price": 142.80, "change": "+0.8%"}
    }
    
    data = stock_data.get(symbol.upper(), {"price": 100.00, "change": "0%"})
    return json.dumps(data)

# Map function names to implementations
functions = {
    "get_weather": get_weather,
    "get_stock_price": get_stock_price
}

print("✅ Functions defined")

In [ ]:
# Create FunctionTool with the Python functions
function_tool = FunctionTool(functions={get_weather, get_stock_price})

# Create toolset
toolset = ToolSet()
toolset.add(function_tool)

# Create agent with function calling
agent_with_functions = project_client.agents.create_agent(
    model=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME", "gpt-4o"),
    name="FunctionAgent",
    instructions="You are a helpful assistant with access to weather and stock price data. Use the functions when needed.",
    tools=toolset.definitions
)

print(f"✅ Agent with functions created: {agent_with_functions.id}")
print("🛠️ Tools: 2 (get_weather, get_stock_price)")

In [ ]:
# Create a new thread
thread_with_functions = project_client.agents.threads.create()

# Add message
message = project_client.agents.messages.create(
    thread_id=thread_with_functions.id,
    role="user",
    content="What's the weather in Seattle and what's Microsoft's stock price?"
)

print(f"📨 Message sent: {message.id}")

# Run the agent with function calling support
run = project_client.agents.runs.create(
    thread_id=thread_with_functions.id,
    agent_id=agent_with_functions.id
)

print("\n🔄 Running agent...")

# Process the run with function calling
while run.status in ["queued", "in_progress", "requires_action"]:
    # Wait for updates
    run = project_client.agents.runs.get(
        thread_id=thread_with_functions.id,
        run_id=run.id
    )

    # Handle function calls
    if run.status == "requires_action":
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        tool_outputs = []

        print(f"\n🔧 Function calls detected: {len(tool_calls)}")

        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_args = json.loads(tool_call.function.arguments)

            print(f"\n📞 Calling: {function_name}({function_args})")

            # Execute the function
            if function_name in functions:
                result = functions[function_name](**function_args)
                print(f"📊 Result: {result}")

                tool_outputs.append({
                    "tool_call_id": tool_call.id,
                    "output": result
                })

        # Submit function results
        project_client.agents.runs.submit_tool_outputs(
            thread_id=thread_with_functions.id,
            run_id=run.id,
            tool_outputs=tool_outputs
        )

print(f"\n✅ Run completed: {run.status}")

# Get the final response
messages = project_client.agents.messages.list(
    thread_id=thread_with_functions.id)

print("\n🤖 Agent Response:")
print("=" * 80)

for msg in messages:
    if msg.role == "assistant":
        if msg.text_messages:
            for text_msg in msg.text_messages:
                print(text_msg.text.value)
        break

print("=" * 80)

## Example 3: Streaming Responses

In [ ]:
# Create a new thread for streaming
thread_streaming = project_client.agents.threads.create()

# Add message
message = project_client.agents.messages.create(
    thread_id=thread_streaming.id,
    role="user",
    content="Explain the benefits of using AI agents in 3 sentences."
)

print("🤖 Streaming Response:")
print("=" * 80)

# Create and stream the run
with project_client.agents.runs.stream(
    thread_id=thread_streaming.id,
    agent_id=agent.id
) as stream:
    for event_type, event_data, _ in stream:
        # Handle text delta events for streaming
        if event_type == "thread.message.delta":
            if hasattr(event_data, "delta") and hasattr(event_data.delta, "content"):
                for content in event_data.delta.content:
                    if hasattr(content, "text") and hasattr(content.text, "value"):
                        print(content.text.value, end='', flush=True)

print("\n" + "=" * 80)
print("✅ Streaming completed")

## Cleanup: Delete Agent and Threads

In [ ]:
# Delete threads
try:
    project_client.agents.threads.delete(thread.id)
    print(f"✅ Thread deleted: {thread.id}")
except Exception as e:
    print(f"⚠️ Could not delete thread: {e}")

try:
    project_client.agents.threads.delete(thread_with_functions.id)
    print(f"✅ Thread deleted: {thread_with_functions.id}")
except Exception as e:
    print(f"⚠️ Could not delete thread: {e}")

try:
    project_client.agents.threads.delete(thread_streaming.id)
    print(f"✅ Thread deleted: {thread_streaming.id}")
except Exception as e:
    print(f"⚠️ Could not delete thread: {e}")

# Delete agents
try:
    project_client.agents.delete_agent(agent.id)
    print(f"✅ Agent deleted: {agent.id}")
except Exception as e:
    print(f"⚠️ Could not delete agent: {e}")

try:
    project_client.agents.delete_agent(agent_with_functions.id)
    print(f"✅ Agent deleted: {agent_with_functions.id}")
except Exception as e:
    print(f"⚠️ Could not delete agent: {e}")

print("\n🧹 Cleanup completed")

## Summary

### What We Covered:

1. **Basic Agent Creation**: Created a simple AI agent with custom instructions
2. **Thread Management**: Created conversation threads to maintain context
3. **Function Calling**: Added custom functions (weather, stock price) to the agent
4. **Streaming**: Implemented real-time streaming responses
5. **Cleanup**: Properly deleted resources after use

### Key Concepts:

- **Agent**: An AI assistant with specific instructions and capabilities
- **Thread**: A conversation session that maintains message history
- **Run**: The execution of an agent on a thread
- **Tools**: Functions that agents can call (weather, stock, file search, code interpreter)

### Next Steps:

- Add tracing decorators to monitor agent performance
- Implement more complex function tools
- Add file search or code interpreter capabilities
- Build a multi-agent system with different specialized agents